# Libraries

In [ ]:
source("tools.R")
library('DescTools')

# Plotting function

In [ ]:
plot_lmls <- function(prop_data, title, real_world_data=c(), hide_x=FALSE, hide_y=FALSE) {
    
    # Real world data
    #points_data <- real_world_data %>% filter(scenario == title)
    #min_pd <- min(points_data$value)
    #max_pd <- max(points_data$value)
    #points_data <- points_data %>% filter(value == max_pd)
    
    plot <- prop_data %>%
                ggplot(aes(x=kernel, y=value)) + 
                    geom_bar(stat="identity") +
                    coord_cartesian (ylim=c(0, 1)) +
                    labs(title = title) +
                    ggthemes::theme_few() +
                    xlab("Kernel composition") + 
                    ylab("Mean likelihood (scaled)") +
                    geom_errorbar(aes(ymin=as.numeric(lo_ci), ymax=as.numeric(hi_ci)),
                                      width=.3,                   
                                      position=position_dodge(.9)) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          text = element_text(size=12, family="serif"),
                          plot.title = element_text(hjust = 0.5))# +
    
                    #Points (real-world data)
                    #geom_point(data=points_data, aes(y=value, x=kernel, group=kernel),
                    #               stat="identity", size=3, color="red", shape=18)
    
    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}

In [ ]:
# Misc functions
scenarios_order <- function (scenarios) {
    scenarios[scenarios=='Temperature'] = 1
    scenarios[scenarios=='Rain']        = 2
    scenarios[scenarios=='Sales']       = 3
    scenarios[scenarios=='Gym members'] = 4
    scenarios[scenarios=='Salary']      = 5
    scenarios[scenarios=='FB friends']  = 6
    
    return(scenarios)
}

# Prior likelihoods

### Preparing the data

In [ ]:
data_prior <- read_csv("data/for_composititional_analysis_prior.csv")

dict_prior <- data_prior %>%                        
                        group_by(id, pid, scenario) %>%
                        summarize()

In [ ]:
lmls_prior <- read_csv("output/lmls_prior.csv")

In [ ]:
metadata_prior <- read_csv("output/metadata_prior.csv")

In [ ]:
metadata_prior <- metadata_prior %>%
                        mutate( cid_composition = paste0(cid, composition) )

metadata_prior$pid <- NULL
metadata_prior$composition <- NULL

kernels <- c("l", "p", "r", "l+p", "l+r", "p+r", "l*r", "l*p", "p*r", "l+r+p", "l+r*p", "l*r+p", "l*p+r", "l*r*p")

lmls_prior<- lmls_prior %>% 
                gather(kernel, lml, kernels)

colnames(lmls_prior) = c('id', 'kernel', 'lml')

lmls_prior <- merge(x = lmls_prior, y = dict_prior, by = c("id", "id"), all.x = TRUE)

lmls_prior <- lmls_prior %>%
                        mutate( cid_composition = paste0(id, kernel) )

# Joining
lmls_prior <- merge(x = lmls_prior, y = metadata_prior, by = c("cid_composition", "cid_composition"), all.x = TRUE)
lmls_prior$cid_composition <- NULL

### Counting
Percentage of GPs that were filtered out (by any reason)

In [ ]:
paste0(round((lmls_prior %>%
            filter(lml == -999999999 | second_exception == 'True') %>%
            filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
            nrow) / 
(lmls_prior %>% 
     filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
     nrow) * 100, 2), '%')

### Summarising and plotting

In [ ]:
# Removes the GPs that failed to be optimized in the second time. The lmls that failed to be optimized are also removed.
#Finaly, the lml_standard is calculated
lmls_prior_f <- lmls_prior %>%
                    filter(second_exception == 'False' & lml != -999999999) %>%
                    group_by(pid, scenario) %>%
                    mutate(lml_minus_min = lml - min(lml),
                           lml_standard = lml_minus_min / (max(lml_minus_min) - min (lml_minus_min)))

sds <- lmls_prior_f %>% 
            group_by(kernel, scenario) %>%
            summarize(lo_bound = mean(lml) - 5*sd(lml), up_bound = mean(lml) + 5*sd(lml))

sds$ks <- paste0(sds$kernel, sds$scenario)

lmls_prior_f$ks <- paste0(lmls_prior_f$kernel, lmls_prior_f$scenario)

sds$kernel <- NULL
sds$scenario <- NULL

lmls_prior_f <- merge(x = lmls_prior_f, y = sds, by = "ks", all.x = TRUE)

In [ ]:
normalize_bool = FALSE
plot_bool      = FALSE

to_plot_prior <- lmls_prior_f %>% 
            filter(lml > -10000) %>%
            filter(lml > lo_bound & lml < up_bound) %>%
            group_by(kernel, scenario) %>%
            summarize(value = mean(lml),
                      lo_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['lwr.ci'],
                      hi_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['upr.ci'])

# Remove single components
to_plot_prior <- to_plot_prior %>%
                filter(kernel != 'l', kernel != 'p', kernel != 'r')

# Scaling the 'data to plot' to a 0-1 range
min_v = min(to_plot_prior$value)
range_v = max(to_plot_prior$value) - min(to_plot_prior$value)

if(normalize_bool) {
    #Normalization
    to_plot_prior <- to_plot_prior %>%
                    group_by( scenario ) %>%
                    mutate( lo_ci = (lo_ci - min(value)) / (max(value) - min(value)),
                            hi_ci = (hi_ci - min(value)) / (max(value) - min(value)),  
                            value = (value - min(value)) / (max(value) - min(value)))
}

# Plotting magic
to_plot_prior$kernel <- factor(to_plot_prior$kernel, levels=kernels)

lmls_temperature <- to_plot_prior %>% filter(scenario == "Temperature")
lmls_rain <- to_plot_prior %>% filter(scenario == "Rain")
lmls_sales <- to_plot_prior %>% filter(scenario == "Sales")
lmls_gym <- to_plot_prior %>% filter(scenario == "Gym members")
lmls_salary <- to_plot_prior %>% filter(scenario == "Salary")
lmls_fb <- to_plot_prior %>% filter(scenario == "FB Friends")

# Real-world data
rwdata <- read_csv("data/real-world/to-plot.csv")

rwdata <- rwdata %>%
    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
    group_by(scenario) %>%
    mutate(value = (lml - min(lml)) / (max(lml) - min(lml)))


if (plot_bool) {
    # Plots
    p1 <- plot_lmls( lmls_temperature, "Temperature", real_world_data=rwdata, hide_x=TRUE)
    p2 <- plot_lmls( lmls_rain,        "Rain",        real_world_data=rwdata, hide_x=TRUE)
    p3 <- plot_lmls( lmls_sales,       "Sales",       real_world_data=rwdata, hide_x=TRUE, hide_y=TRUE)
    p4 <- plot_lmls( lmls_gym,         "Gym members", real_world_data=rwdata, hide_x=TRUE, hide_y=TRUE)
    p5 <- plot_lmls( lmls_salary,      "Salary",      real_world_data=rwdata, hide_x=TRUE, hide_y=TRUE)
    p6 <- plot_lmls( lmls_fb,          "FB Friends",  real_world_data=rwdata, hide_x=TRUE, hide_y=TRUE)

    pdf("Images/paper_images/kernels_priors_lmls_5sd_over-10k.pdf", width=8, height=4)
    multiplot(p1, p2, p3, p4, p5, p6, cols=3)
    dev.off()   
}

### Prior condition. Best performing kernel compositions:

In [ ]:
to_plot_prior %>%
    group_by(scenario) %>%
    summarize(kernel[which.max(value)], value[which.max(value)]) %>%
    arrange(scenarios_order(scenario))

In [ ]:
# Which other kernel compositions are statistically equivalent
to_plot_prior %>%
    group_by(scenario) %>%
    filter(hi_ci > lo_ci[which.max(value)]) %>%
    select(scenario, kernel, value, lo_ci, hi_ci) %>%
    arrange(scenarios_order(scenario))

# Full-Bayesian Posterior analysis

### Preparing the data

In [ ]:
lmls_posterior <- read_csv("output/full-bayesian-posterior/results_posterior_test_lmls.csv")

In [ ]:
# To add the 'scenario' column

data_posterior <- read_csv("data/for_composititional_analysis_posterior.csv")

dict_posterior <- data_posterior %>%                        
                        group_by(id, pid, scenario) %>%
                        summarize()

dict_posterior$pid <- NULL

lmls_posterior <- merge(x = lmls_posterior, y = dict_posterior, by = c("id", "id"), all.x = TRUE)

# Adding 'participant id' (pid)
lmls_posterior$pid <- floor((lmls_posterior$id-1) / 6) + 1

### Analyzing

In [ ]:
print('White added')

lmls_posterior %>%
    filter(white_added == 'TRUE') %>%
    nrow

lmls_posterior %>%
    filter(white_added == 'FALSE') %>%
    nrow

lmls_posterior %>%
    nrow

#######

print('Second exception')
lmls_posterior %>%
    filter(second_exception == 'TRUE') %>%
    nrow

lmls_posterior %>%
    filter(second_exception == 'FALSE') %>%
    nrow

lmls_posterior %>%
    nrow

######

print('Proportion of LML error and Second exception over the total')

paste0(round((lmls_posterior %>%
            filter(lml == -999999999 | second_exception == 'TRUE') %>% 
            filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
            nrow) / (lmls_posterior %>% 
                     filter(kernel != 'l', kernel != 'p', kernel != 'r') %>%
                     nrow) * 100, 2), '%')

In [ ]:
# Removing unusable data, and standardizing.
lmls_posterior_f <- lmls_posterior %>%
                    filter(second_exception == 'FALSE' & lml != -999999999) %>%
                    group_by(pid, scenario) %>%
                    mutate(lml_minus_min = lml - min(lml),
                           lml_standard = lml_minus_min / (max(lml_minus_min) - min (lml_minus_min)))

# Standard deviations calculation
sds <- lmls_posterior_f %>% 
            group_by(kernel, scenario) %>%
            summarize(lo_bound = mean(lml) - 5*sd(lml), up_bound = mean(lml) + 5*sd(lml))

sds$ks <- paste0(sds$kernel, sds$scenario)

lmls_posterior_f$ks <- paste0(lmls_posterior_f$kernel, lmls_posterior_f$scenario)

sds$kernel <- NULL
sds$scenario <- NULL

lmls_posterior_f <- merge(x = lmls_posterior_f, y = sds, by = "ks", all.x = TRUE)

### Plotting

In [ ]:
normalize_bool = FALSE
plot_bool      = FALSE

# Plotting data
to_plot_posterior <- lmls_posterior_f %>%
            filter(lml > -10000) %>%
            filter(lml > lo_bound & lml < up_bound) %>%
            group_by(kernel, scenario) %>%
            summarize(value = mean(lml),
                      lo_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['lwr.ci'],
                      hi_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['upr.ci'])

# Remove single components
to_plot_posterior <- to_plot_posterior %>%
                filter(kernel != 'l', kernel != 'p', kernel != 'r')

# Scaling the 'data to plot' to a 0-1 range
min_v = min(to_plot_posterior$value)
range_v = max(to_plot_posterior$value) - min(to_plot_posterior$value)

if(normalize_bool){
    # Normalization
    to_plot_posterior <- to_plot_posterior %>%
                    group_by( scenario ) %>%
                    mutate( lo_ci = (lo_ci - min(value)) / (max(value) - min(value)),
                            hi_ci = (hi_ci - min(value)) / (max(value) - min(value)),  
                            value = (value - min(value)) / (max(value) - min(value)))  
}

# Plotting magic
to_plot_posterior$kernel <- factor(to_plot_posterior$kernel, levels=kernels)

lmls_temperature <- to_plot_posterior %>% filter(scenario == "Temperature")
lmls_rain <- to_plot_posterior %>% filter(scenario == "Rain")
lmls_sales <- to_plot_posterior %>% filter(scenario == "Sales")
lmls_gym <- to_plot_posterior %>% filter(scenario == "Gym members")
lmls_salary <- to_plot_posterior %>% filter(scenario == "Salary")
lmls_fb <- to_plot_posterior %>% filter(scenario == "FB Friends")

if (plot_bool){
    p1 <- plot_lmls(lmls_temperature, "Temperature", hide_x=TRUE)
    p2 <- plot_lmls(lmls_rain, "Rain", hide_x=TRUE)
    p3 <- plot_lmls(lmls_sales, "Sales", hide_x=TRUE, hide_y=TRUE)
    p4 <- plot_lmls(lmls_gym, "Gym members", hide_y=TRUE, hide_x=TRUE)
    p5 <- plot_lmls(lmls_salary, "Salary", hide_x=TRUE, hide_y=TRUE)
    p6 <- plot_lmls(lmls_fb, "FB Friends", hide_y=TRUE, hide_x=TRUE)

    pdf("Images/paper_images/kernels_posteriors_lmls_5sd.pdf", width=8, height=4)
    multiplot(p1, p2, p3, p4, p5, p6, cols=3)
    dev.off()
}

In [ ]:
# Which kernel is max
to_plot_posterior %>%
    group_by(scenario) %>%
    summarize(kernel[which.max(value)], value[which.max(value)], lo_ci[which.max(value)]) %>%
    arrange(scenarios_order(scenario))

In [ ]:
# Which other kernel compositions are statistically equivalent
to_plot_posterior %>%
    group_by(scenario) %>%
    filter(hi_ci > lo_ci[which.max(value)]) %>%
    select(scenario, kernel, value, lo_ci, hi_ci) %>%
    arrange(scenario) %>%
    arrange(scenarios_order(scenario))

# Best on each condition

In [ ]:
lmls_prior_ff <- lmls_prior_f %>%
                    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>% 
                    filter(lml > lo_bound & lml < up_bound) %>%
                    filter(lml > -10000)

lmls_posterior_ff <- lmls_posterior_f %>%
                    filter(kernel != 'l', kernel != 'p', kernel != 'r') %>% 
                    filter(lml > lo_bound & lml < up_bound) %>%
                    filter(lml > -10000)

### Averages

In [ ]:
prior_a <- lmls_prior_ff %>%
                    group_by(kernel, scenario) %>%
                    summarize(value = mean(lml),
                              lo_ci = t.test(lml, conf.level=0.95)$conf.int[1],
                              hi_ci = t.test(lml, conf.level=0.95)$conf.int[2])

posterior_a <- lmls_posterior_ff %>%
                    group_by(kernel, scenario) %>%
                    summarize(value = mean(lml),
                              lo_ci = t.test(lml, conf.level=0.95)$conf.int[1],
                              hi_ci = t.test(lml, conf.level=0.95)$conf.int[2])

### Tables

In [ ]:
prior_a %>% 
    group_by(scenario) %>%
    summarize(max_index = which.max(value),
              max_kernel = kernel[max_index],
              max_lml = value[max_index]) %>%
    arrange(scenarios_order(scenario))

In [ ]:
posterior_a %>% 
    group_by(scenario) %>%
    summarize(max_index = which.max(value),
              max_kernel = kernel[max_index],
              max_lml = value[max_index]) %>%
    arrange(scenarios_order(scenario))

### Consistency

In [ ]:
prior_maxs <- lmls_prior_f %>%
                        filter(kernel != 'l', kernel != 'p', kernel != 'r') %>% 
                        group_by(id, pid, scenario) %>%
                        summarize(max_index = which.max(lml),
                                  max_kernel = kernel[max_index],
                                  max_lml = lml[max_index])

In [ ]:
posterior_maxs <- lmls_posterior_f %>%
                        filter(kernel != 'l', kernel != 'p', kernel != 'r') %>% 
                        group_by(id, pid, scenario) %>%
                        summarize(max_index = which.max(lml),
                                  max_kernel = kernel[max_index],
                                  max_lml = lml[max_index])

In [ ]:
merged_maxs <- merge(x = prior_maxs, y = posterior_maxs, by = c("id", "id")) #inner join

Percentage of consistency (match):

In [ ]:
merged_maxs <- merged_maxs %>%
    mutate(match = (max_kernel.x == max_kernel.y))

In [ ]:
(( merged_maxs %>%
     filter(match == TRUE) %>%
     nrow) /
 ( merged_maxs %>%
     nrow) ) %>% round(2)

Chance:

In [ ]:
(1/11) %>% round(2)

Bootstrapped confidence interval:

In [ ]:
bp = function(x, lev, n = 1e6, alpha=0.05) {
  res = replicate(n, sum(sample(x, length(x), replace=TRUE) == lev)/length(x))
  return(list(mean=mean(res),
              `95% CI`=quantile(res, c(0.5*alpha,1-0.5*alpha))))
}

In [ ]:
bp2 = function(vector) {
    
    match_ci <- bp(vector$match, TRUE)
    
    return( c( match_ci$mean %>% round(2),
             match_ci$`95% CI`[1] %>% round(2),
             match_ci$`95% CI`[2] %>% round(2) ))
}

In [ ]:
bp2(merged_maxs)

### Consistency per POSTERIOR condition

In [ ]:
#Importing the relevant data
aux <- read_csv("data/for_composititional_analysis_posterior.csv")

posterior_conditions <- aux %>% select(id, condition) %>% unique

# Merging
merged_maxs <- merge(x = merged_maxs, y = posterior_conditions, by = c("id", "id")) 

In [ ]:
bp3 = function(condition) {
    print(condition)
    merged_maxs %>%
        filter(condition == condition) %>%
        bp2 %>%
        print
}

In [ ]:
print('All')
print(bp2(merged_maxs))

print('Posterior-Positive')
    merged_maxs %>%
        filter(condition == 'Posterior-Positive') %>%
        bp2 %>%
        print

print('Posterior-Stable')
    merged_maxs %>%
        filter(condition == 'Posterior-Stable') %>%
        bp2 %>%
        print

print('Posterior-Negative')
    merged_maxs %>%
        filter(condition == 'Posterior-Negative') %>%
        bp2 %>%
        print

In [ ]:
bp3 = function(data, filter_v, condition = TRUE){
    
    if(condition){
        results <- data %>%
                    filter(condition == !!filter_v) %>%
                    bp2 
    }
    else {
        results <- data %>%
                    filter(scenario.x == !!filter_v) %>%
                    bp2 
    }
    
    data.frame(list( condition = filter_v, 
                     mean  = results[[1]], 
                     lo_ci = results[[2]],
                     hi_ci = results[[3]]))
}

In [ ]:
plot_consistence = function(consistence_data, title, xlab, ylim_hi=0.45, ylab="Proportion of Match")
    consistence_data %>%
                ggplot(aes(x=condition, y=mean)) + 
                    geom_bar(stat="identity", aes(fill = fill)) +
                    scale_fill_manual(values=c("grey35", "black")) +
                    coord_cartesian (ylim=c(0, ylim_hi)) +
                    labs(title = title) +
                    ggthemes::theme_few() +
                    theme(legend.position="none") +
                    xlab(xlab) + 
                    ylab(ylab) +
                    geom_errorbar(aes(ymin=as.numeric(lo_ci), ymax=as.numeric(hi_ci)),
                                      width=.3,                   
                                      position=position_dodge(.9)) +
                    theme(#axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          text = element_text(size=12, family="serif"),
                          plot.title = element_text(hjust = 0.5)) +
                    geom_hline(yintercept = 1/11, linetype="dashed") #chance value

In [ ]:
chance <- data.frame(list(condition="Chance", mean=1/11, lo_ci=1/13, hi_ci=1/12)) #the confidence interval is set in lower values for it to disappear

consistence_condition <- rbind( #chance,
                                bp3(merged_maxs, "Posterior-Positive"),
                                bp3(merged_maxs, "Posterior-Stable"),
                                bp3(merged_maxs, "Posterior-Negative"))

consistence_scenario <- rbind(  #chance,
                                bp3(merged_maxs, "Temperature", FALSE),
                                bp3(merged_maxs, "Rain", FALSE),
                                bp3(merged_maxs, "Sales", FALSE),
                                bp3(merged_maxs, "Gym members", FALSE),
                                bp3(merged_maxs, "Salary", FALSE),
                                bp3(merged_maxs, "FB Friends", FALSE))

In [ ]:
#Extend the text so the plots match in size
longer_text <- "FB Friends             "

levels(consistence_scenario$condition) <- c(levels(consistence_scenario$condition), longer_text)

#consistence_scenario[7,1] <- longer_text

In [ ]:
# Add this for colors
consistence_condition$fill <- (consistence_condition$condition == "Chance")
consistence_scenario$fill <- (consistence_scenario$condition == "Chance")

In [ ]:
# Readable names of Posterior conditions
levels(consistence_condition$condition) <- c(levels(consistence_condition$condition), "Positive", "Stable", "Negative")

# Replacement
consistence_condition[consistence_condition$condition == "Posterior-Positive", "condition"] <- "Positive"
consistence_condition[consistence_condition$condition == "Posterior-Stable", "condition"] <- "Stable"
consistence_condition[consistence_condition$condition == "Posterior-Negative", "condition"] <- "Negative"

In [ ]:
# Readable Gym
levels(consistence_scenario$condition) <- c(levels(consistence_scenario$condition), "Gym")
consistence_scenario[consistence_scenario$condition == "Gym members", "condition"] <- "Gym"

levels(consistence_scenario$condition) <- c(levels(consistence_scenario$condition), "FB friends")
consistence_scenario[consistence_scenario$condition == "FB Friends", "condition"] <- "FB friends"

In [ ]:
plot_cond <- consistence_condition %>% plot_consistence(title="(a)", xlab="Posterior conditions")

plot_scen <- consistence_scenario %>% plot_consistence(title="(b)", xlab="Scenarios")

pdf("Images/paper_images/match_proportion.pdf", width=4, height=6)
multiplot(plot_cond, plot_scen, cols=1)
dev.off()  

# Match index

In [ ]:
# Which other kernel compositions are statistically equivalent
best_prior <- to_plot_prior %>%
    group_by(scenario) %>%
    filter(hi_ci > lo_ci[which.max(value)]) %>%
    select(scenario, kernel, value, lo_ci, hi_ci) %>%
    arrange(scenario)

# Which other kernel compositions are statistically equivalent
best_posterior <- to_plot_posterior %>%
    group_by(scenario) %>%
    filter(hi_ci > lo_ci[which.max(value)]) %>%
    select(scenario, kernel, value, lo_ci, hi_ci) %>%
    arrange(scenario)

In [ ]:
best_prior %>% head

In [ ]:
# Inner join
inner_aux <- merge(x = best_prior, y = best_posterior, by.x = c('scenario', 'kernel'), by.y = c('scenario', 'kernel')) %>%
                    group_by(scenario) %>%
                    summarize(count = length(kernel))

In [ ]:
# Outer join
outer_aux <- merge(x = best_prior, y = best_posterior, by.x = c('scenario', 'kernel'), by.y = c('scenario', 'kernel'), all.x=TRUE, all.y=TRUE) %>%
                    group_by(scenario) %>%
                    summarize(count = length(kernel))

In [ ]:
merge(inner_aux, outer_aux, by='scenario') %>%
    mutate(match = round(count.x/count.y, 2))

# Ranking correlation

In [ ]:
# This is the participants data, in the Prior condition. 
# We are filtering LML lower than -10.000, and out of the 5 standardad deviations. Failed GPs have already been filtered
lmls_prior_f %>%
            filter(lml > -10000) %>%
            filter(lml > lo_bound & lml < up_bound) %>%
            head

In [304]:
# Real world data
rwdata <- read_csv("data/real-world/to-plot.csv")

Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
